asm-mc_v1.1
修正点は以下
<br>・def dest内 ifがずれていて一部動作しなかったのを修正
<br>・def code内 containsへうまく渡せず、一致しててもTrueを返さなかったのを修正
<br>・def code内 変数用のiの値が+1ずつ増加するよう修正

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
def fileopen():
  with open('/content/gdrive/MyDrive/3S_PBL/Pong.asm') as f:
    list = f.readlines()
  return list

In [ ]:
def hasMoreCommands(l):
  t=l.lstrip(' ')
  if t[0]=='/':
    return False
  elif t[0]=='\n':
    return False
  else:
    return True

In [ ]:
def advance(l):
  t=l.lstrip(' ')
  command=t
  return command

In [ ]:
def commandType(command):
  if command[0]=='@':
    return 'A_COMMAND'
  elif command[0]=='(':
    return 'L_COMMAND'
  else:
    return 'C_COMMAND'

In [ ]:
def symbol(l):
  if l[0]=='@':
    t=l.lstrip('@')
    t2=t.rstrip('\n')
    return t2

In [ ]:
def dest(l):
  if l[0]=='D':
    if l[1]=='=':
      return 'D'
    else:
      return 'null'

  elif l[0]=='M':
    if l[1]=='=':
      return 'M'

    elif l[1]=='D':
      if l[2]=='=':
        return 'MD'
      else:
        return 'null'

  elif l[0]=='A':
    if l[1]=='=':
      return 'A'

    elif l[1]=='D':
      if l[2]=='=':
        return 'AD'

    elif l[1]=='M':
      if l[2]=='=':
        return 'AM'

      elif l[2]=='D':
        if l[3]=='=':
          return 'AMD'
        else:
          return 'null'
      else:
        return 'null'
  else:
    return 'null'

In [ ]:
def comp(l,ds):
  if ds=='null':
    t=l
  else:
    tmp=l.lstrip(ds)
    t=tmp.lstrip('=')

  if t[0]=='0':
    return '0'

  elif t[0]=='1':
    return '1'

  elif t[0]=='!':
    if t[1]=='D':
      return '!D'

    elif t[1]=='A':
      return '!A'

    elif t[1]=='M':
      return '!M'

  elif t[0]=='-':
    if t[1]=='1':
      return '-1'

    elif t[1]=='D':
      return '-D'

    elif t[1]=='A':
      return '-A'

    elif t[1]=='M':
      return '-M'

  elif t[0]=='A':
    if t[1]=='+':
      return 'A+1'

    elif t[1]=='-':
      if t[2]=='1':
        return 'A-1'

      elif t[2]=='D':
        return 'A-D'

    else:
      return 'A'

  elif t[0]=='D':
    if t[1]=='+':
      if t[2]=='1':
        return 'D+1'

      elif t[2]=='A':
        return 'D+A'

      elif t[2]=='M':
        return 'D+M'

    elif t[1]=='-':
      if t[2]=='1':
        return 'D-1'

      elif t[2]=='A':
        return 'D-A'

      elif t[2]=='M':
        return 'D-M'

    elif t[1]=='&':
      if t[2]=='A':
        return 'D&A'

      elif t[2]=='M':
        return 'D&M'

    elif t[1]=='|':
      if t[2]=='A':
        return 'D|A'

      elif t[2]=='M':
        return 'D|M'

    else:
      return 'D'

  elif t[0]=='M':
    if t[1]=='+':
      return 'M+1'

    elif t[1]=='-':
      if t[2]=='1':
        return 'M-1'

      elif t[2]=='D':
        return 'M-D'

    else:
      return 'M'

In [ ]:
def jump(l,ds,cm):
  if cm=='null':
    t=l
  else:
    tmp=l.lstrip(ds)
    tmp2=tmp.lstrip('=')
    tmp3=tmp2.lstrip(cm)
    tmp4=tmp3.lstrip(';')
    tmp5=tmp4.rstrip('\n')
    t=tmp5.rstrip(' ')
  if t=='JGT':
    return 'JGT'

  elif t=='JEQ':
    return 'JEQ'

  elif t=='JGE':
    return 'JGE'

  elif t=='JLT':
    return 'JLT'

  elif t=='JNE':
    return 'JNE'

  elif t=='JLE':
    return 'JLE'

  elif t=='JMP':
    return 'JMP'

  else:
    return 'null'


In [ ]:
def createSBTable():
  sbtable=[]
  sbtable.append(['SP',0])
  sbtable.append(['LCL',1])
  sbtable.append(['ARG',2])
  sbtable.append(['THIS',3])
  sbtable.append(['THAT',4])
  sbtable.append(['R0',0])
  sbtable.append(['R1',1])
  sbtable.append(['R2',2])
  sbtable.append(['R3',3])
  sbtable.append(['R4',4])
  sbtable.append(['R5',5])
  sbtable.append(['R6',6])
  sbtable.append(['R7',7])
  sbtable.append(['R8',8])
  sbtable.append(['R9',9])
  sbtable.append(['R10',10])
  sbtable.append(['R11',11])
  sbtable.append(['R12',12])
  sbtable.append(['R13',13])
  sbtable.append(['R14',14])
  sbtable.append(['R15',15])
  sbtable.append(['SCREEN',16384])
  sbtable.append(['KBD',24576])
  return sbtable

In [ ]:
def Parser():
  i=0
  list=fileopen()
  outlist=[]
  sbtable=createSBTable()
  for l in list:
    if hasMoreCommands(l):
      l=advance(l)
      ct=commandType(l)
      if ct=='A_COMMAND':
        sb=symbol(l)
        ds='null'
        cm='null'
        jp='null'
        i+=1
      elif ct=='C_COMMAND':
        sb='null'
        ds=dest(l)
        cm=comp(l,ds)
        jp=jump(l,ds,cm)
        i+=1
      elif ct=='L_COMMAND':
        sb='null'
        ds='null'
        cm='null'
        jp='null'
        t=l.lstrip('(')
        t2=t.rstrip('\n')
        t3=t2.rstrip(')')
        sbtable.append([t3,i])
    else:
      sb='null'
      ds='null'
      cm='null'
      jp='null'
    outlist.append([sb,ds,cm,jp])
  return(outlist,sbtable)

In [ ]:
def Cdest(ds):
  if ds=='null':
    return('000')
  elif ds=='M':
    return('001')
  elif ds=='D':
    return('010')
  elif ds=='MD':
    return('011')
  elif ds=='A':
    return('100')
  elif ds=='AM':
    return('101')
  elif ds=='AD':
    return('110')
  elif ds=='AMD':
    return('111')
  else:
    return('000')

In [ ]:
def Ccomp(cm):
  if cm=='0':
    return('0101010')
  elif cm=='1':
    return('0111111')
  elif cm=='-1':
    return('0111010')
  elif cm=='D':
    return('0001100')
  elif cm=='A':
    return('0110000')
  elif cm=='!D':
    return('0001101')
  elif cm=='!A':
    return('0110001')
  elif cm=='-D':
    return('0001111')
  elif cm=='-A':
    return('0110011')
  elif cm=='D+1':
    return('0011111')
  elif cm=='A+1':
    return('0110111')
  elif cm=='D-1':
    return('0001110')
  elif cm=='A-1':
    return('0110010')
  elif cm=='D+A':
    return('0000010')
  elif cm=='D-A':
    return('0010011')
  elif cm=='A-D':
    return('0000111')
  elif cm=='D&A':
    return('0000000')
  elif cm=='D|A':
    return('0010101')
  elif cm=='M':
    return('1110000')
  elif cm=='!M':
    return('1110001')
  elif cm=='-M':
    return('1110011')
  elif cm=='M+1':
    return('1110111')
  elif cm=='M-1':
    return('1110010')
  elif cm=='D+M':
    return('1000010')
  elif cm=='D-M':
    return('1010011')
  elif cm=='M-D':
    return('1000111')
  elif cm=='D&M':
    return('1000000')
  elif cm=='D|M':
    return('1010101')
  else:
    return('000')

In [ ]:
def Cjump(jp):
  if jp=='null':
    return('000')
  elif jp=='JGT':
    return('001')
  elif jp=='JEQ':
    return('010')
  elif jp=='JGE':
    return('011')
  elif jp=='JLT':
    return('100')
  elif jp=='JNE':
    return('101')
  elif jp=='JLE':
    return('110')
  elif jp=='JMP':
    return('111')
  else:
    return('000')

In [ ]:
def addEntry(l,i,sbtable):
  sbtable.append([l,i])
  return sbtable

In [ ]:
def contains(symbol,sbtable):
  for sb in sbtable:
    if sb[0]==symbol:
      return True
  return False

In [ ]:
def getAddress(symbol,sbtable):
  for sb in sbtable:
    if sb[0]==symbol:
      return sb[1]

In [ ]:
def Symboltable(symbol,sbtable):
  if contains(symbol,sbtable):
    return getAddress(symbol,sbtable)
  else:
    sbtable.append([symbol])

In [ ]:
def Code():
  list,sbtable=Parser()
  outlist=[]
  i=16
  for l in list:
    if l[0]=='null':
      if l[2]!='null':
        sb='111'
        ds=Cdest(l[1])
        cm=Ccomp(l[2])
        jp=Cjump(l[3])
        len=sb+cm+ds+jp
        outlist.append([len])
    else:
      if l[0].isdigit():
        num=int(l[0])
        bin = '0'+format(num & 0xFFFF, '015b')
        outlist.append([bin])
      else:
        if contains(l[0],sbtable):
          bin = '0'+format(getAddress(l[0],sbtable) & 0xFFFF, '015b')
          outlist.append([bin])
        else:
          sbtable=addEntry(l[0],i,sbtable)
          bin = '0'+format(getAddress(l[0],sbtable) & 0xFFFF, '015b')
          outlist.append([bin])
          i+=1
  return(outlist)

In [ ]:
def main():
  list=Code()
  for l in list:
    print(l)
  with open('/content/gdrive/MyDrive/3S_PBL/Pong1.hack', 'w', encoding='utf-8') as f:
    for item in list:
      f.write(item[0]+'\n')

In [ ]:
t,s=Parser()
for l in t:
  print(l)

ストリーミング出力は最後の 5000 行に切り捨てられました。
['null', 'AM', 'M-1', 'null']
['null', 'D', 'M', 'null']
['null', 'A', 'A-1', 'null']
['null', 'M', 'M-D', 'null']
['LCL', 'null', 'null', 'null']
['null', 'D', 'M', 'null']
['8', 'null', 'null', 'null']
['null', 'D', 'D+A', 'null']
['R13', 'null', 'null', 'null']
['null', 'M', 'D', 'null']
['SP', 'null', 'null', 'null']
['null', 'AM', 'M-1', 'null']
['null', 'D', 'M', 'null']
['R13', 'null', 'null', 'null']
['null', 'A', 'M', 'null']
['null', 'M', 'D', 'null']
['LCL', 'null', 'null', 'null']
['null', 'D', 'M', 'null']
['7', 'null', 'null', 'null']
['null', 'A', 'D+A', 'null']
['null', 'D', 'M', 'null']
['SP', 'null', 'null', 'null']
['null', 'AM', 'M+1', 'null']
['null', 'A', 'A-1', 'null']
['null', 'M', 'D', 'null']
['screen.0', 'null', 'null', 'null']
['null', 'D', 'M', 'null']
['SP', 'null', 'null', 'null']
['null', 'AM', 'M+1', 'null']
['null', 'A', 'A-1', 'null']
['null', 'M', 'D', 'null']
['SP', 'null', 'null', 'null']
['null', 'AM', 'M-1', 'null'

In [ ]:
main()

ストリーミング出力は最後の 5000 行に切り捨てられました。
['1110110010100000']
['1110001100001000']
['0101011111011001']
['1110110000010000']
['0000000000010110']
['1110101010000111']
['0000000000000000']
['1111110010101000']
['1111110000010000']
['1110110010100000']
['1111010101001000']
['0000000000000000']
['1111110010101000']
['1111110000010000']
['0101011111100101']
['1110001100000101']
['0101011111111100']
['1110101010000111']
['0000000000001001']
['1110110000010000']
['0000000000000000']
['1111110111101000']
['1110110010100000']
['1110001100001000']
['0000000000000001']
['1110110000010000']
['0000000000001101']
['1110001100001000']
['0110101011011001']
['1110110000010000']
['0000000000001110']
['1110001100001000']
['0101011111110111']
['1110110000010000']
['0000000001011111']
['1110101010000111']
['0000000000000000']
['1111110010101000']
['1111110000010000']
['0000000000000101']
['1110001100001000']
['0000000000000010']
['1111110000100000']
['1111110000010000']
['0000000000000000']
['1111110111101000']
['